# Multithreading and Multiprocessing

Recall the phrase "many hands make light work". This is as true in programming as anywhere else.

What if you could engineer your Python program to do four things at once? What would normally take an hour could (almost) take one fourth the time.<font color=green>\*</font>

This is the idea behind parallel processing, or the ability to set up and run multiple tasks concurrently.


<br><font color=green>\* *We say almost, because you do have to take time setting up four processors, and it may take time to pass information between them.*</font>

##  Threading vs. Processing

From a computational point of view, multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. On the other hand, multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.

From the diagram below, we can see that in multithreading (middle diagram), multiple threads share the same code, data and files but run on a different register and stack. Multiprocessing (the right diagram) multiplies a single processor, replicating the code, data and files, which incurs more overhead.

![title](https://builtin.com/sites/www.builtin.com/files/styles/ckeditor_optimize/public/inline-images/1_multithreading-vs-multiprocessing.jpg)

Multithreading is useful for IO-bound processes, such as reading files from a network or database since each thread can run the IO-bound process concurrently. Multiprocessing is useful for CPU-bound processes, such as computationally heavy tasks since it will benefit from having multiple processors; similar to how multicore computers work faster than computers with a single core.

There is a difference between concurrency and parallelism. Parallelism allows multiple tasks to execute at the same time, whereas concurrency allows multiple tasks to execute one at a time in an interleaving manner.

Due to Python global interpreter lock (GIL), only one thread can be executed at a time. Therefore, multithreading only achieves concurrency and not parallelism for IO-bound processes. On the other hand, multiprocessing achieves parallelism.

Using multithreading for CPU-bound processes might slow down performance due to competing resources that ensure only one thread can execute at a time, and overhead is incurred in dealing with multiple threads.

On the other hand, multiprocessing can be used for IO-bound processes. However, the overhead for managing multiple processes is higher than managing multiple threads as illustrated above. You may notice that multiprocessing might lead to higher CPU utilization due to multiple CPU cores being used by the program, which is expected.

#### I/O-intensive processes improved with multithreading:
* webscraping
* reading and writing to files
* sharing data between programs
* network communications


#### CPU-intensive processes improved with multiprocessing:
* computations
* text formatting
* image rescaling
* data analysis

## Multithreading

A thread is a separate flow of execution. This means that your program will have two things happening at once. But for most Python 3 implementations the different threads do not actually execute at the same time: they merely appear to.

It’s tempting to think of threading as having two (or more) different processors running on your program, each one doing an independent task at the same time. That’s almost right. The threads may be running on different processors, but they will only be running one at a time.

Getting multiple tasks running simultaneously requires a non-standard implementation of Python, writing some of your code in a different language, or using multiprocessing which comes with some extra overhead.

Now that you’ve got an idea of what a thread is, let’s learn how to make one. The Python standard library provides `threading`, which contains most of the primitives you’ll see in this article. `Thread`, in this module, nicely encapsulates threads, providing a clean interface to work with them.

To start a separate thread, you create a `Thread` instance and then tell it to `.start()`:

In [1]:
%%writefile thread_example.py
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    #####
    logging.info("Main    : all done")

Overwriting thread_example.py


In [2]:
! python thread_example.py

10:12:31: Main    : before creating thread
10:12:31: Main    : before running thread
10:12:31: Thread 1: starting
10:12:31: Main    : wait for the thread to finish
10:12:31: Main    : all done
10:12:33: Thread 1: finishing


You’ll notice that the Thread finished after the Main section of your code did. If you want to wait for a thread to stop you must use the `.join()` function.

In [3]:
%%writefile thread_example.py
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    x.join()
    logging.info("Main    : all done")

Overwriting thread_example.py


In [4]:
! python thread_example.py

10:12:34: Main    : before creating thread
10:12:34: Main    : before running thread
10:12:34: Thread 1: starting
10:12:34: Main    : wait for the thread to finish
10:12:36: Thread 1: finishing
10:12:36: Main    : all done


### Daemon threads

In computer science, a `daemon` is a process that runs in the background.

Python threading has a more specific meaning for daemon. A daemon thread will shut down immediately when the program exits. One way to think about these definitions is to consider the daemon thread a thread that runs in the background without worrying about shutting it down.

If a program is running Threads that are not daemons, then the program will wait for those threads to complete before it terminates. Threads that are daemons, however, are just killed wherever they are when the program is exiting.<br><br>

Let’s look a little more closely at the output of your program above. The last two lines are the interesting bit. When you run the program, you’ll notice that there is a pause (of about 2 seconds) after `__main__` has printed its all done message and before the thread is finished.

This pause is Python waiting for the non-daemonic thread to complete. When your Python program ends, part of the shutdown process is to clean up the threading routine.

If you look at the source for Python threading, you’ll see that `threading._shutdown()` walks through all of the running threads and calls `.join()` on every one that does not have the daemon flag set.

So your program waits to exit because the thread itself is waiting in a sleep. As soon as it has completed and printed the message, `.join()` will return and the program can exit.

Frequently, this behavior is what you want, but there are other options available to us. Let’s first repeat the program with a daemon thread. You do that by changing how you construct the Thread, adding the `daemon=True` flag:

In [5]:
%%writefile daemon.py
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,), daemon=True)
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    #####
    logging.info("Main    : all done")

Overwriting daemon.py


In [6]:
! python daemon.py

10:12:36: Main    : before creating thread
10:12:36: Main    : before running thread
10:12:36: Thread 1: starting
10:12:36: Main    : wait for the thread to finish
10:12:36: Main    : all done


The difference here is that the final line of the output is missing. `thread_function()` did not get a chance to complete. It was a daemon thread, so when `__main__` reached the end of its code and the program wanted to finish, the daemon was killed.

### Working with many threads

The easier way to start a group of threads in Python is through a `ThreadPoolExecutor`, and it’s part of the standard library in `concurrent.futures` (as of Python 3.2).

In [7]:
%%writefile multithreading.py
import logging
import time
import concurrent.futures

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(thread_function, range(3))

Overwriting multithreading.py


In [8]:
! python multithreading.py

10:12:36: Thread 0: starting
10:12:36: Thread 1: starting
10:12:36: Thread 2: starting
10:12:38: Thread 0: finishing
10:12:38: Thread 1: finishing
10:12:38: Thread 2: finishing


Note: The scheduling of threads is done by the operating system and does not follow a plan that’s easy to figure out.

### Example: Race Condition

Race conditions can occur when two or more threads access a shared piece of data or resource. In this example, you’re going to create a large race condition that happens every time, but be aware that most race conditions are not this obvious. Frequently, they only occur rarely, and they can produce confusing results. As you can imagine, this makes them quite difficult to debug.

For this example, we will considering updating a fake database with two different threads. To make the race condition always happen, we use the `time.sleep(0.1)` call, delaying the execution.

In [9]:
%%writefile fakedb.py
import threading
import logging
import time
import concurrent.futures

class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    database = FakeDatabase()
    logging.info("Testing update. Starting value is %d.", database.value)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for index in range(2):
            executor.submit(database.update, index)
    logging.info("Testing update. Ending value is %d.", database.value)

Overwriting fakedb.py


In [10]:
! python fakedb.py

10:12:38: Testing update. Starting value is 0.
10:12:38: Thread 0: starting update
10:12:38: Thread 1: starting update
10:12:38: Thread 0: finishing update
10:12:38: Thread 1: finishing update
10:12:38: Testing update. Ending value is 1.


As you can see, even if the `update()` function was called by both threads, but the final result that was "saved" is 1 instead of 2.

One solution to this problem is through Locks (or mutex for other programming languages). Locks can be defined using the `threading.Lock()` function

In [11]:
%%writefile fakedb.py
import threading
import logging
import time
import concurrent.futures

class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def locked_update(self, name):
        logging.info("Thread %s: starting update", name)
        logging.debug("Thread %s: about to lock", name)
        with self._lock:
            logging.debug("Thread %s: has lock", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.debug("Thread %s: about to release lock", name)
        logging.debug("Thread %s: after release", name)
        logging.info("Thread %s: finishing update", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.DEBUG,
                        datefmt="%H:%M:%S")

    database = FakeDatabase()
    logging.info("Testing update. Starting value is %d.", database.value)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for index in range(2):
            executor.submit(database.locked_update, index)
    logging.info("Testing update. Ending value is %d.", database.value)

Overwriting fakedb.py


In [12]:
! python fakedb.py

10:12:39: Testing update. Starting value is 0.
10:12:39: Thread 0: starting update
10:12:39: Thread 1: starting update
10:12:39: Thread 0: about to lock
10:12:39: Thread 1: about to lock
10:12:39: Thread 0: has lock
10:12:39: Thread 0: about to release lock
10:12:39: Thread 0: after release
10:12:39: Thread 0: finishing update
10:12:39: Thread 1: has lock
10:12:39: Thread 1: about to release lock
10:12:39: Thread 1: after release
10:12:39: Thread 1: finishing update
10:12:39: Testing update. Ending value is 2.


### Example: Producer-Consumer threading

The [Producer-Consumer Problem](https://en.wikipedia.org/wiki/Producer%E2%80%93consumer_problem) is a standard computer science problem used to look at threading or process synchronization issues.

Imagine a program that needs to read messages from a network and write them to disk. The program does not request a message when it wants. It must be listening and accept messages as they come in. The messages will not come in at a regular pace, but will be coming in bursts. This part of the program is called the __producer__.

On the other side, once you have a message, you need to write it to a database. The database access is slow, but fast enough to keep up to the average pace of messages. It is not fast enough to keep up when a burst of messages comes in. This part is the __consumer__.

Producer and consumer communicate through a __pipeline__, which can be implemented in multiple ways. If you want to be able to handle more than one value in the pipeline at a time, you’ll need a data structure for the pipeline that allows the number to grow and shrink as data backs up from the producer. The best fit for this is a `Queue`, provided by Python with the `queue` module.

Additionally, the implementation we present makes use of the `threading.Event` object. This allows one thread to signal an event while many other threads can be waiting for that event to happen. The key usage in this code is that the threads that are waiting for the event do not necessarily need to stop what they are doing, they can just check the status of the Event every once in a while.

In [13]:
%%writefile multithreading.py
import concurrent.futures
import logging
import queue
import random
import threading
import time

def producer(queue, event):
    """Pretend we're getting a number from the network."""
    while not event.is_set():
        message = random.randint(1, 101)
        logging.info("Producer got message: %s", message)
        queue.put(message)

    logging.info("Producer received event. Exiting")

def consumer(queue, event):
    """Pretend we're saving a number in the database."""
    while not event.is_set() or not queue.empty():
        message = queue.get()
        logging.info(
            "Consumer storing message: %s (size=%d)", message, queue.qsize()
        )

    logging.info("Consumer received event. Exiting")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    pipeline = queue.Queue(maxsize=10)
    event = threading.Event()
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(producer, pipeline, event)
        executor.submit(consumer, pipeline, event)

        time.sleep(0.1)
        logging.info("Main: about to set event")
        event.set()

Overwriting multithreading.py


In [14]:
! python multithreading.py

10:12:39: Producer got message: 43
10:12:39: Producer got message: 84
10:12:39: Consumer storing message: 43 (size=0)
10:12:39: Producer got message: 52
10:12:39: Consumer storing message: 84 (size=0)
10:12:39: Consumer storing message: 52 (size=0)
10:12:39: Producer got message: 66
10:12:39: Producer got message: 2
10:12:39: Producer got message: 6
10:12:39: Consumer storing message: 66 (size=1)
10:12:39: Consumer storing message: 2 (size=1)
10:12:39: Consumer storing message: 6 (size=0)
10:12:39: Producer got message: 100
10:12:39: Producer got message: 89
10:12:39: Consumer storing message: 100 (size=0)
10:12:39: Producer got message: 95
10:12:39: Consumer storing message: 89 (size=0)
10:12:39: Producer got message: 88
10:12:39: Consumer storing message: 95 (size=0)
10:12:39: Producer got message: 41
10:12:39: Consumer storing message: 88 (size=0)
10:12:39: Producer got message: 96
10:12:39: Consumer storing message: 41 (size=0)
10:12:39: Producer got message: 50
10:12:39: Consumer 

## Multiprocessing

In Python, the `multiprocessing` module includes an API for dividing work between multiple processes. To initiate a process you have to instantiate the `Process` class, similarly to the threading class

In [15]:
%%writefile test_processing.py
# importing the multiprocessing module 
import multiprocessing 
import os 
  
def worker1(): 
    # printing process id 
    print("ID of process running worker1: {}".format(os.getpid())) 
  
def worker2(): 
    # printing process id 
    print("ID of process running worker2: {}".format(os.getpid())) 
  
if __name__ == "__main__": 
    # printing main program process id 
    print("ID of main process: {}".format(os.getpid())) 
  
    # creating processes 
    p1 = multiprocessing.Process(target=worker1) 
    p2 = multiprocessing.Process(target=worker2) 
  
    # starting processes 
    p1.start() 
    p2.start() 
  
    # process IDs 
    print("ID of process p1: {}".format(p1.pid)) 
    print("ID of process p2: {}".format(p2.pid)) 
  
    # wait until processes are finished 
    p1.join() 
    p2.join() 
  
    # both processes finished 
    print("Both processes finished execution!") 
  
    # check if processes are alive 
    print("Process p1 is alive: {}".format(p1.is_alive())) 
    print("Process p2 is alive: {}".format(p2.is_alive())) 

Writing test_processing.py


In [16]:
! python test_processing.py

ID of main process: 557
ID of process p1: 558
ID of process p2: 559
ID of process running worker1: 558
ID of process running worker2: 559
Both processes finished execution!
Process p1 is alive: False
Process p2 is alive: False


In multiprocessing, any newly created process will:
- run independently
- have their own memory space

### Locking and Pooling

To handle synchronization between processes, the `multiprocessing` module also provides his own `Lock()` class to deal with race conditions. Let's see it in action for a basic implementation of a bank deposit/withdrawal system

In [17]:
%%writefile bank.py
import multiprocessing 
  
# function to withdraw from account 
def withdraw(balance, lock):     
    for _ in range(10000): 
        lock.acquire() 
        balance.value = balance.value - 1
        lock.release() 
  
# function to deposit to account 
def deposit(balance, lock):     
    for _ in range(10000): 
        lock.acquire() 
        balance.value = balance.value + 1
        lock.release() 
  
def perform_transactions(): 
  
    # initial balance (in shared memory) 
    balance = multiprocessing.Value('i', 100) 
  
    # creating a lock object 
    lock = multiprocessing.Lock() 
  
    # creating new processes 
    p1 = multiprocessing.Process(target=withdraw, args=(balance,lock)) 
    p2 = multiprocessing.Process(target=deposit, args=(balance,lock)) 
  
    # starting processes 
    p1.start() 
    p2.start() 
  
    # wait until processes are finished 
    p1.join() 
    p2.join() 
  
    # print final balance 
    print("Final balance = {}".format(balance.value)) 
  
if __name__ == "__main__": 
    for _ in range(10): 
  
        # perform same transaction process 10 times 
        perform_transactions() 

Writing bank.py


In [18]:
! python bank.py

Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100
Final balance = 100


Additionally, the module also provides a `Pool` class. The `Pool` class represents a pool of worker processes. It has methods which allows tasks to be offloaded to the worker processes in a few different ways. In a multi-core/multi-processor system, tasks are offloaded/distributed among the cores/processes automatically by Pool object. Hence, an user doesn’t need to worry about creating processes explicitly. Let's see it in action.

In [19]:
%%writefile pooling.py
import multiprocessing 
import os 
  
def square(n): 
    print("Worker process id for {0}: {1}".format(n, os.getpid())) 
    return (n*n) 
  
if __name__ == "__main__": 
    # input list 
    mylist = [1,2,3,4,5] 
  
    # creating a pool object 
    p = multiprocessing.Pool() 
  
    # map list to target function 
    result = p.map(square, mylist) 
  
    print(result) 

Writing pooling.py


In [20]:
! python pooling.py

Worker process id for 1: 582
Worker process id for 2: 583
Worker process id for 3: 584
Worker process id for 4: 585
Worker process id for 5: 586
[1, 4, 9, 16, 25]


### Example: Monte Carle Method and Estimating Pi

Let's code out an example to see how the parts fit together. We can time our results using the *timeit* module to measure any performance gains. Our task is to apply the Monte Carlo Method to estimate the value of Pi.

If you draw a circle of radius 1 (a unit circle) and enclose it in a square, the areas of the two shapes are given as

<table>
    <caption>Area Formulas</caption>
    <tr><td>circle</td><td> πr^2 </td></tr>
    <tr><td>square</td><td>4 r^2</td></tr>
</table>


Therefore, the ratio of the volume of the circle to the volume of the square is $$\frac{π}{4}$$

The Monte Carlo Method plots a series of random points inside the square. By comparing the number that fall within the circle to those that fall outside, with a large enough sample we should have a good approximation of Pi. You can see a good demonstration of this [here](https://academo.org/demos/estimating-pi-monte-carlo/) (Hit the **Animate** button on the page).

For a given number of points *n*, we have $$π = \frac{4 \cdot points\ inside\ circle}{total\ points\ n}$$

To set up our multiprocessing program, we first derive a function for finding Pi that we can pass to `map()`:

In [21]:
from random import random  # perform this import outside the function

def find_pi(n):
    """
    Function to estimate the value of Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # if i falls inside the circle
            inside+=1

    pi=4*inside/n
    return pi

Let's test `find_pi` on 5,000 points:

In [22]:
find_pi(5000)

3.1256

This ran very quickly, but the results are not very accurate!

Next we'll write a script that sets up a pool of workers, and lets us time the results against varying sized pools. We'll set up two arguments to represent *processes* and *total_iterations*. Inside the script, we'll break *total_iterations* down into the number of iterations passed to each process, by making a processes-sized list.<br>For example:

    total_iterations = 1000
    processes = 5
    iterations = [total_iterations//processes]*processes
    iterations
    # Output: [200, 200, 200, 200, 200]
    
This list will be passed to our `map()` function along with `find_pi()`

In [23]:
%%writefile montecarlo.py
from random import random
from multiprocessing import Pool
import timeit

def find_pi(n):
    """
    Function to estimate the value of Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # if i falls inside the circle
            inside+=1

    pi=4*inside/n
    return pi

if __name__ == '__main__':
    N = 10**5  # total iterations
    P = 5      # number of processes
    
    p = Pool(P)
    print(timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.7f}'), number=10))
    p.close()
    p.join()
    print(f'{N} total iterations with {P} processes')

Writing montecarlo.py


In [24]:
! python montecarlo.py

3.1501600
3.1364000
3.1430800
3.1401600
3.1417600
3.1436400
3.1385600
3.1438000
3.1409600
3.1379600
0.338676729999861
100000 total iterations with 5 processes


Great! The above test took under a second on our computer.

Now that we know our script works, let's increase the number of iterations, and compare two different pools. Sit back, this may take awhile!

In [25]:
%%writefile montecarlo.py
from random import random
from multiprocessing import Pool
import timeit

def find_pi(n):
    """
    Function to estimate the value of Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # if i falls inside the circle
            inside+=1

    pi=4*inside/n
    return pi

if __name__ == '__main__':
    N = 10**7  # total iterations
    
    P = 1      # number of processes
    p = Pool(P)
    print(timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.7f}'), number=10))
    p.close()
    p.join()
    print(f'{N} total iterations with {P} processes')
    
    P = 5      # number of processes
    p = Pool(P)
    print(timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.7f}'), number=10))
    p.close()
    p.join()
    print(f'{N} total iterations with {P} processes')

Overwriting montecarlo.py


In [26]:
! python montecarlo.py

3.1418856
3.1416112
3.1412780
3.1419700
3.1416976
3.1418280
3.1423628
3.1424536
3.1413008
3.1420452
28.121344073999353
10000000 total iterations with 1 processes
3.1410108
3.1414348
3.1412844
3.1417300
3.1406648
3.1417404
3.1422920
3.1418920
3.1417708
3.1414896
21.727108373999727
10000000 total iterations with 5 processes


Hopefully you saw that with 5 processes our script ran faster!

#### More is Better ...to a point.

The gain in speed as you add more parallel processes tends to flatten out at some point. In any collection of tasks, there are going to be one or two that take longer than average, and no amount of added processing can speed them up. This is best described in [Amdahl's Law](https://en.wikipedia.org/wiki/Amdahl%27s_law).

#### A final script

In the example below, we'll add a context manager to shrink these three lines

    p = Pool(P)
    ...
    p.close()
    p.join()
    
to one line:

    with Pool(P) as p:
    
And we'll accept command line arguments using the *sys* module.
    

In [27]:
%%writefile montecarlo2.py
from random import random
from multiprocessing import Pool
import timeit
import sys

N = int(sys.argv[1])  # these arguments are passed in from the command line
P = int(sys.argv[2])

def find_pi(n):
    """
    Function to estimate the value of Pi
    """
    inside=0

    for i in range(0,n):
        x=random()
        y=random()
        if (x*x+y*y)**(0.5)<=1:  # if i falls inside the circle
            inside+=1
    pi=4*inside/n
    return pi

if __name__ == '__main__':
    
    with Pool(P) as p:
        print(timeit.timeit(lambda: print(f'{sum(p.map(find_pi, [N//P]*P))/P:0.5f}'), number=10))
    print(f'{N} total iterations with {P} processes')

Writing montecarlo2.py


In [28]:
! python montecarlo2.py 10000000 500

3.14185
3.14236
3.14161
3.14126
3.14191
3.14133
3.14180
3.14113
3.14128
3.14191
36.36400427499939
10000000 total iterations with 500 processes


Great! Now you should have a good understanding of multithreading and multiprocessing!